# Demo 05 - Merge, Join, and SQL

In this demo we will first see how to do merges and joins in Pandas and then try them with an instance of SQLite

In [1]:
# first, mount your google drive, change to the course folder, pull latest changes, and change to the lab folder.
# Startup Magic to: (1) Mount Google Drive
# (2) Change to Course Folder
# (3) Pull latest Changes
# (4) Move to the Demo Directory so that the data files are available

from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/cmps3160
!git pull
%cd _demos

Mounted at /content/drive
/content/drive/My Drive/cmps3160
Already up to date.
/content/drive/My Drive/cmps3160/_demos


In [2]:
# Includes and Standard Magic...
### Standard Magic and startup initializers.

# Load Numpy
import numpy as np
# Load MatPlotLib
import matplotlib
import matplotlib.pyplot as plt
# Load Pandas
import pandas as pd

# This lets us show plots inline and also save PDF plots if we want them
%matplotlib inline
from matplotlib.backends.backend_pdf import PdfPages
matplotlib.style.use('fivethirtyeight')

# These two things are for Pandas, it widens the notebook and lets us display data easily.
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

# Show a ludicrus number of rows and columns
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## Joins in Pandas

Let's see a few [joins in Pandas](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html) and how to execute them.

**Note:** We can also use the [merge command in pandas](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html) to joins as well.  We'll explore this more fully in the lab.  The key idea is that merge let's us pick columns to do the merge/join while join will always join on the index.

In [3]:
# Careful here! We have to set the index!

df1 = pd.DataFrame([(1, 'foo', 3),
                    (2, 'bar', 2),
                    (3, 'foo', 4),
                    (4, 'foo', 3)],
                  columns=['ID','A', 'B'])
df1.set_index('ID', inplace=True)
display(df1)

df2 = pd.DataFrame([(1, 1.2), (2, 2.5), (3, 2.3), (5, 8.0)], 
                   columns=['ID','C'])
df2.set_index('ID', inplace=True)
display(df2)

,A,B
ID,,
1,foo,3
2,bar,2
3,foo,4
4,foo,3


,C
ID,
1,1.2
2,2.5
3,2.3
5,8.0


In [4]:
# Inner join
df1.join(df2, how='inner')

,A,B,C
ID,,,
1,foo,3,1.2
2,bar,2,2.5
3,foo,4,2.3


In [5]:
# Can do inner, outer, left, right.
df1.join(df2, how='right')

,A,B,C
ID,,,
1,foo,3.0,1.2
2,bar,2.0,2.5
3,foo,4.0,2.3
5,NaN,NaN,8.0


If we don't have the same column names we can tell Pandas that we want to join on a specific column.

In [6]:
df1 = pd.DataFrame([(1, 'foo', 3), (2, 'bar', 2), (3, 'foo', 4), (4, 'foo', 3)],
                  columns=['leftid','A', 'B'])
display(df1)

df2 = pd.DataFrame([(1, 1.2), (2, 2.5), (3, 2.3), (5, 8.0)], 
                   columns=['rightid','C'])
df2.set_index('rightid', inplace=True)
display(df2)

,leftid,A,B
0,1,foo,3
1,2,bar,2
2,3,foo,4
3,4,foo,3


,C
rightid,
1,1.2
2,2.5
3,2.3
5,8.0


In [7]:
# Inner join
df1.join(df2, how='inner')
# Is this what we wanted?!?!

,leftid,A,B,C
1,2,bar,2,1.2
2,3,foo,4,2.5
3,4,foo,3,2.3


In [8]:
# We could have set the index but we can also tell Pandas to do it for us.
df1.join(df2, how='inner', on='leftid')

,leftid,A,B,C
0,1,foo,3,1.2
1,2,bar,2,2.5
2,3,foo,4,2.3


## Pandas Merge

Or we can skip all this foolishness by using the [pandas merge](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html) command...

In [9]:
df1 = pd.DataFrame([(1, 'foo', 3), (2, 'bar', 2), (3, 'foo', 4), (4, 'foo', 3)],
                  columns=['ID','A', 'B'])
display(df1)

df2 = pd.DataFrame([(1, 1.2), (2, 2.5), (3, 2.3), (5, 8.0)], 
                   columns=['ID','C'])
display(df2)

,ID,A,B
0,1,foo,3
1,2,bar,2
2,3,foo,4
3,4,foo,3


,ID,C
0,1,1.2
1,2,2.5
2,3,2.3
3,5,8.0


In [10]:
df1.merge(df2, left_on="ID", right_on="ID", how='inner')

,ID,A,B,C
0,1,foo,3,1.2
1,2,bar,2,2.5
2,3,foo,4,2.3


In [11]:
df1.merge(df2, left_on="ID", right_on="ID", how='outer')

,ID,A,B,C
0,1,foo,3.0,1.2
1,2,bar,2.0,2.5
2,3,foo,4.0,2.3
3,4,foo,3.0,NaN
4,5,NaN,NaN,8.0


In [12]:
df1.merge(df2, left_on="ID", right_on="ID", how='left')

,ID,A,B,C
0,1,foo,3,1.2
1,2,bar,2,2.5
2,3,foo,4,2.3
3,4,foo,3,NaN


In [13]:
df1.merge(df2, left_on="ID", right_on="ID", how='right')

,ID,A,B,C
0,1,foo,3.0,1.2
1,2,bar,2.0,2.5
2,3,foo,4.0,2.3
3,5,NaN,NaN,8.0


## Using SQL with SQLite!

Helpful here is the [SQLite Cheat Sheet](https://www.sqlitetutorial.net/sqlite-cheat-sheet/) and [Python library](https://docs.python.org/3/library/sqlite3.html) documentation.

In [14]:
import pandas as pd
import sqlite3

In [15]:
!pwd

/content/drive/My Drive/cmps3160/_demos


In [16]:
# Create a DB and connect to it.
# These creates a new file to store the database.
conn = sqlite3.connect("lec09-n.db")
cursor = conn.cursor()

In [17]:
cursor.execute('''
CREATE TABLE cats (
    cat_id INTEGER PRIMARY KEY,
    name TEXT
)''')


In [18]:
# Insert into the table
cursor.execute('''INSERT INTO cats VALUES (1, "Megabyte")''')
cursor.execute('''INSERT INTO cats VALUES (2, "Meowly Cyrus")''')
cursor.execute('''INSERT INTO cats VALUES (3, "Fuzz Aldrin")''')
cursor.execute('''INSERT INTO cats VALUES (4, "Chairman Meow")''')
cursor.execute('''INSERT INTO cats VALUES (5, "Anderson Pooper")''')
cursor.execute('''INSERT INTO cats VALUES (6, "Gigabyte")''')
# Commit will update the database on disk with a set of changes.
conn.commit()


In [19]:
# What if we run an SQL Command?
cursor.execute('''SELECT * FROM cats''')

In [20]:
# We need to iterate over the object we get back...
for row in cursor.execute('''SELECT * FROM cats'''):
    print(row)

(1, 'Megabyte')
(2, 'Meowly Cyrus')
(3, 'Fuzz Aldrin')
(4, 'Chairman Meow')
(5, 'Anderson Pooper')
(6, 'Gigabyte')


In [21]:
# Or we can just go to Pandas from here...  
# note that we can tell Pandas what to do for an index here.
pd.read_sql_query("SELECT * FROM cats", conn, index_col="cat_id")

,name
cat_id,
1,Megabyte
2,Meowly Cyrus
3,Fuzz Aldrin
4,Chairman Meow
5,Anderson Pooper
6,Gigabyte


In [22]:
# This is a full up DB so we can delete stuff too..
cursor.execute('''DELETE FROM cats WHERE cat_id == 4''')
conn.commit()
pd.read_sql_query("SELECT * FROM cats", conn, index_col="cat_id")

,name
cat_id,
1,Megabyte
2,Meowly Cyrus
3,Fuzz Aldrin
5,Anderson Pooper
6,Gigabyte


In [23]:
# Let's make another table for joins.
cursor.execute('''
CREATE TABLE visits (
    visit_id INTEGER PRIMARY KEY,
    date DATE,
    cat_id INTEGER,
    FOREIGN KEY(cat_id) REFERENCES cats(cat_id)
)''')

In [24]:
cursor.execute('''INSERT INTO visits VALUES (1, "02-16-2017", 1)''')
cursor.execute('''INSERT INTO visits VALUES (2, "02-14-2017", 1)''')
cursor.execute('''INSERT INTO visits VALUES (5, "02-03-2017", 2)''')
conn.commit()

In [25]:
# Check what we did...
pd.read_sql_query("SELECT * FROM visits", conn, index_col="visit_id")

,date,cat_id
visit_id,,
1,02-16-2017,1
2,02-14-2017,1
5,02-03-2017,2


# Joins in SQL

Note that the SELECT command can give us an inner join by default and we don't have to use the keyword INNER JOIN.

In [26]:
# We can do our join in Pandas
df_cats = pd.read_sql_query('''SELECT * from cats''', conn)
display(df_cats)
df_visits = pd.read_sql_query('''SELECT * from visits''', conn)
display(df_visits)

,cat_id,name
0,1,Megabyte
1,2,Meowly Cyrus
2,3,Fuzz Aldrin
3,5,Anderson Pooper
4,6,Gigabyte


,visit_id,date,cat_id
0,1,02-16-2017,1
1,2,02-14-2017,1
2,5,02-03-2017,2


In [27]:
# Here we bring down all the data then do our join..
df_cats.merge(df_visits, how = "inner", 
              left_on = "cat_id", right_on = "cat_id")

,cat_id,name,visit_id,date
0,1,Megabyte,1,02-16-2017
1,1,Megabyte,2,02-14-2017
2,2,Meowly Cyrus,5,02-03-2017


In [28]:
# Or we can directly run them on the system

result = cursor.execute('''
               SELECT 
                   *
               FROM 
                   cats, visits
               WHERE
                   cats.cat_id == visits.cat_id
               ''')
for i in result:
    print(i)


(1, 'Megabyte', 1, '02-16-2017', 1)
(1, 'Megabyte', 2, '02-14-2017', 1)
(2, 'Meowly Cyrus', 5, '02-03-2017', 2)


In [29]:
# We can also use union to combine rows, even if it doesn't make sense.
result = cursor.execute('''
               SELECT 
                   *
               FROM 
                   cats
               UNION
               SELECT
                   cat_id, date
               FROM
                   visits
               ''')
for i in result:
    print(i)

(1, '02-14-2017')
(1, '02-16-2017')
(1, 'Megabyte')
(2, '02-03-2017')
(2, 'Meowly Cyrus')
(3, 'Fuzz Aldrin')
(5, 'Anderson Pooper')
(6, 'Gigabyte')


In [30]:
# What if we want a different join?
df_cats.merge(df_visits, how = "left", 
              left_on = "cat_id", right_on = "cat_id")



,cat_id,name,visit_id,date
0,1,Megabyte,1.0,02-16-2017
1,1,Megabyte,2.0,02-14-2017
2,2,Meowly Cyrus,5.0,02-03-2017
3,3,Fuzz Aldrin,NaN,NaN
4,5,Anderson Pooper,NaN,NaN
5,6,Gigabyte,NaN,NaN


In [31]:
result = cursor.execute('''SELECT * FROM 
                                cats 
                           LEFT JOIN 
                               visits 
                           ON
                               cats.cat_id == visits.cat_id''')
for i in result:
    print(i)

(1, 'Megabyte', 2, '02-14-2017', 1)
(1, 'Megabyte', 1, '02-16-2017', 1)
(2, 'Meowly Cyrus', 5, '02-03-2017', 2)
(3, 'Fuzz Aldrin', None, None, None)
(5, 'Anderson Pooper', None, None, None)
(6, 'Gigabyte', None, None, None)


In [32]:
# What if we want a different join?
df_cats.merge(df_visits, how = "outer", 
              left_on = "cat_id", right_on = "cat_id")



,cat_id,name,visit_id,date
0,1,Megabyte,1.0,02-16-2017
1,1,Megabyte,2.0,02-14-2017
2,2,Meowly Cyrus,5.0,02-03-2017
3,3,Fuzz Aldrin,NaN,NaN
4,5,Anderson Pooper,NaN,NaN
5,6,Gigabyte,NaN,NaN


In [33]:
# Let's do an Outer Join!

result = cursor.execute('''SELECT * FROM cats FULL OUTER JOIN visits 
                    ON cats.cat_id == visits.cat_id''')
for i in result:
    print(i)

OperationalError: ignored

So we have to be careful as SQLite has some limitations... but you get the idea.


## GroupBy and Aggregate in SQL.


In [34]:
# Make a new Table...
cursor.execute('''
CREATE TABLE persons (
    id INTEGER PRIMARY KEY,
    age FLOAT,
    wgt_kg FLOAT,
    hgt_cm FLOAT,
    nat_id INTEGER
    
)''')

In [35]:
# Insert into the table
cursor.execute('''INSERT INTO persons VALUES (1, 12.2, 42.3, 145.1, 1)''')
cursor.execute('''INSERT INTO persons VALUES (2, 11.0, 40.8, 143.8, 1)''')
cursor.execute('''INSERT INTO persons VALUES (3, 15.6, 65.3, 165.3, 2)''')
cursor.execute('''INSERT INTO persons VALUES (4, 35.1, 84.2, 185.8, 1)''')
cursor.execute('''INSERT INTO persons VALUES (5, 18.1, 62.2, 176.2, 3)''')
cursor.execute('''INSERT INTO persons VALUES (6, 19.6, 82.1, 180.1, 1)''')
conn.commit()

In [36]:
ret = cursor.execute('''SELECT * FROM persons''')
for row in ret:
    print(row)

(1, 12.2, 42.3, 145.1, 1)
(2, 11.0, 40.8, 143.8, 1)
(3, 15.6, 65.3, 165.3, 2)
(4, 35.1, 84.2, 185.8, 1)
(5, 18.1, 62.2, 176.2, 3)
(6, 19.6, 82.1, 180.1, 1)


In [37]:
ret = cursor.execute('''SELECT nat_id, wgt_kg, AVG(age) as average_age
                        FROM persons
                        GROUP BY nat_id''')
for row in ret:
    print(row)

(1, 82.1, 19.475)
(2, 65.3, 15.6)
(3, 62.2, 18.1)


### Make sure you close your connection or things go badly!!

- Writes all the changes to the database.

In [38]:
conn.close()

### More practice in SQL

That about does it for the SQL for this class -- You can do the [SQL murder mystery for bonus!](http://mystery.knightlab.com/walkthrough.html)